In [1]:
import pandas as pd
from pandas import read_excel

file_name = 'target_data.xlsx' 
target_data = read_excel(file_name)
target_data = target_data.fillna(0)

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
feature_cols = list(target_data.columns[1:-6])

X_all = target_data[feature_cols]

In [3]:
from sklearn.ensemble import RandomForestClassifier
import pickle

mfmodel = pickle.load(open('mfmodel.pkl', 'rb'))

prediction = mfmodel.predict(X_all)
sales_cols = list(target_data.columns[:-6])
result = target_data[sales_cols]
result["mf_sales_prediction"] = prediction
result = result[result.mf_sales_prediction == 1]
print(result.shape[0])
print(result[:1000])
result.to_excel("mf_sales_prediction.xlsx", index=False)

207
      Client  Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  \
0          1  1.0   51       7         1       0.0       0.0        1.0   
26        27  0.0   62     181         1       0.0       0.0        1.0   
33        34  1.0   48     104         1       1.0       3.0        1.0   
35        36  1.0   49     263         1       1.0       2.0        0.0   
41        42  0.0   23       4         1       0.0      12.0        0.0   
...      ...  ...  ...     ...       ...       ...       ...        ...   
1563    1564  1.0   29     175         1       0.0       0.0        1.0   
1568    1569  1.0    3     152         1       3.0      32.0        0.0   
1570    1571  0.0   39     211         1       0.0       0.0        0.0   
1584    1585  1.0   28     221         1       0.0       0.0        0.0   
1612    1613  0.0   46      45         1       0.0       0.0        0.0   

      Count_CC  Count_CL  ...  VolumeDeb_CA  VolumeDebCash_Card  \
0          0.0       0.0  ..

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [4]:
result

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,mf_sales_prediction
0,1,1.0,51,7,1,0.0,0.0,1.0,0.0,0.0,...,1335.619643,250.000000,283.089286,247.607143,50.0,30.0,3.0,12.0,9.0,1
26,27,0.0,62,181,1,0.0,0.0,1.0,0.0,0.0,...,67.857143,0.000000,0.000000,0.000000,2.0,1.0,0.0,0.0,0.0,1
33,34,1.0,48,104,1,1.0,3.0,1.0,1.0,0.0,...,7683.438214,1107.142857,844.575357,5775.094286,54.0,19.0,6.0,33.0,11.0,1
35,36,1.0,49,263,1,1.0,2.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1
41,42,0.0,23,4,1,0.0,12.0,0.0,0.0,0.0,...,546.642857,178.571429,68.535714,296.071429,9.0,9.0,1.0,1.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,1564,1.0,29,175,1,0.0,0.0,1.0,0.0,0.0,...,506.002857,139.285714,122.081786,83.928571,41.0,27.0,6.0,14.0,1.0,1
1568,1569,1.0,3,152,1,3.0,32.0,0.0,0.0,0.0,...,596.742857,0.000000,422.350000,171.000000,20.0,20.0,0.0,13.0,6.0,1
1570,1571,0.0,39,211,1,0.0,0.0,0.0,0.0,0.0,...,2.678571,0.000000,0.000000,0.000000,2.0,2.0,0.0,0.0,0.0,1
1584,1585,1.0,28,221,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1
